In [4]:
!pip install requests

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.0-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (34 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
Using cached charset_normalizer-3.4.0-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (144 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)


In [1]:
import requests

In [2]:
endpoint= "http://api.citybik.es//v2/networks/bicicorunha"
requests.get(endpoint)


<Response [200]>

In [7]:
!pip install pymongo
!pip install schedule

  Using cached pymongo-4.10.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (22 kB)
  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
Using cached pymongo-4.10.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.2 MB)
Using cached dnspython-2.7.0-py3-none-any.whl (313 kB)
  Using cached schedule-1.2.2-py3-none-any.whl.metadata (3.8 kB)
Using cached schedule-1.2.2-py3-none-any.whl (12 kB)


In [ ]:
import requests
import pymongo
import schedule
import time
from datetime import datetime

def get_mongo_client():
    try:
        client = pymongo.MongoClient("mongodb://localhost:27017/")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Error de conexión con MongoDB: {e}")
        return None

client = get_mongo_client()
if client:
    db = client["bicicorunha_db"]
    collection = db["data"]
else:
    print("No se pudo conectar a MongoDB.")
    exit(1)

def fetch_and_store_data():
    url = "http://api.citybik.es/v2/networks/bicicorunha"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Lanza un erro se a resposta non é exitosa
        data = response.json()

        # Engadimos a data e hora
        data['timestamp'] = datetime.now().isoformat()

        # Gardamos os datos en MongoDB
        collection.insert_one(data)
        print(f"Datos guardados correctamente en MongoDB: {data['timestamp']}")
    except requests.exceptions.RequestException as e:
        print(f"Error al conectar a la API: {e}")
    except Exception as e:
        print(f"Error al guardar los datos en MongoDB: {e}")

# Programamos a execución a intervalos de 3 minutos
schedule.every(3).minutes.do(fetch_and_store_data)

# Mantemos o script en execución
try:
    while True:
        schedule.run_pending()  # Executa as tarefas programadas se é necesario
        time.sleep(1)  # Espera 1 segundo antes de comprobar de novo
except KeyboardInterrupt:
    # Aquí manejamos la desconexión manual
    print("\nDesconexión manual detectada. El script fue detenido correctamente.")



Desconexión manual detectada. El script fue detenido correctamente.


In [5]:
import pandas as pd
import pymongo

def get_mongo_client():
    try:
        client = pymongo.MongoClient("mongodb://localhost:27017/")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Error de conexión con MongoDB: {e}")
        return None

client = get_mongo_client()
if client:
    db = client["bicicorunha_db"]
    collection = db["data"]
else:
    print("No se pudo conectar a MongoDB, saíndo.")
    exit(1)

try:
    documents = collection.find()

    data = []
    for doc in documents:
        for station in doc['network']['stations']:
            data.append({
                "id": station.get("id"),
                "name": station.get("name"),
                "timestamp": doc.get("timestamp"),
                "free_bikes": station.get("free_bikes"),
                "empty_slots": station.get("empty_slots"),
                "uid": station.get("extra", {}).get("uid"),
                "last_updated": station.get("extra", {}).get("last_updated"),
                "slots": station.get("extra", {}).get("slots"),
                "normal_bikes": station.get("extra", {}).get("normal_bikes"),
                "ebikes": station.get("extra", {}).get("ebikes")
            })

    df = pd.DataFrame(data)

    if not df.empty:
        csv_filename = "bicicorunha_data.csv"
        df.to_csv(csv_filename, index=False)
        print(f"Datos exportados correctamente a {csv_filename}")

        parquet_filename = "bicicorunha_data.parquet"
        df.to_parquet(parquet_filename, index=False)
        print(f"Datos exportados correctamente a {parquet_filename}")
    else:
        print("No se encontraron datos para exportar.")

except Exception as e:
    print(f"Error al procesar los datos: {e}")

Datos exportados correctamente a bicicorunha_data.csv
Datos exportados correctamente a bicicorunha_data.parquet


In [ ]:
version: '3.8'

services:
  mongo:
    image: mongo:latest
    container_name: mongoBicis
    ports:
      - "27017:27017"  
    networks:
      - bici_network
    volumes:
      - mongo_data:/data/db 

  scripts:
    build:
      context: ./scripts  
    container_name: script_runner
    networks:
      - bici_network
    volumes:
      - ./scripts:/usr/src/app  
    environment:
      MONGO_HOST: mongoBicis
      MONGO_PORT: 27017